In [11]:
import sys
sys.path.append("..")
from utils.audio_utils import get_total_audio_duration,get_audio_info,get_duration_ffprobe
from pathlib import Path
from tqdm import tqdm
from utils.file_utils import save_json
from utils.transcription_utils import clean_and_verify_transcript_hi
from collections import Counter
from utils.conversions import sec_to_time
import csv

In [12]:
save_path=Path("/media/dataset-harddisk/munikumar/utils/audio_utils/data/common_voice/")

In [13]:
dataset_dir=Path("/media/dataset-harddisk/munikumar/hindi_dataset/common_voice/cv-corpus-13.0-2023-03-09/hi")


In [14]:
def get_transcription_info(transcription_path,dataset_path):
    print(f"getting audio info about : {transcription_path.stem}")
    non_hindi_chars = []
    non_hindi_transcriptions = {}
    total_duration=0
    file_count=0
    with open(transcription_path, "r", encoding="utf-8") as fp:
        reader=csv.reader(fp,delimiter='\t')
        next(reader)
        transcription_data = {
            transcription[1]:transcription[2]
            for transcription in reader
        }
    audio_data=[]
    min_duration=1e9
    max_duration=-1e9
    total_duration=0
    audio_file_duration={}
    for audio_file, transcription in tqdm(transcription_data.items()):
        audio=dataset_path/audio_file
        data=get_audio_info(str(audio))
        audio_data.append(data)
        duration=float(data["duration"])
        max_duration=max(duration,max_duration)
        min_duration=min(duration,min_duration)
        total_duration+=duration
        audio_file_duration[audio.name]=duration
        clean_transcription, non_hindi_tokens = clean_and_verify_transcript_hi(transcription)
        if len(non_hindi_tokens) != 0:
            file_count+=1
            duration=get_duration_ffprobe(dataset_path/audio_file)
            non_hindi_transcriptions[audio_file] = {
                "raw_transcription": transcription,
                "clean_transcription": clean_transcription,
                "duration":duration,
                "non_hindi_chars":non_hindi_tokens
            }
            total_duration+=duration
            
            non_hindi_chars.extend(non_hindi_tokens)

    audio_file_duration["max_duration"]=max_duration
    audio_file_duration['min_duration']=min_duration
    audio_file_duration['total_duration']=total_duration
    save_json(audio_data,save_path=save_path/f"common_voice_{transcription_path.stem}_info.json")
    save_json(audio_file_duration,save_path=save_path/f"common_voice_{transcription_path.stem}_duration.json")

    print(non_hindi_chars)
    non_hindi_char_freq=dict(Counter(non_hindi_chars))
    non_hindi_transcriptions["total_audio_duration"]=total_duration
    non_hindi_transcriptions["file_count"]=file_count
    print("file_count",file_count)
    save_json(non_hindi_transcriptions,save_path=save_path/f"common_voice_{transcription_path.stem}_non_hindi_transcripts.json",ensure_ascii=False)
    save_json(non_hindi_char_freq,save_path=save_path/f"common_voice_{transcription_path.stem}_non_hindi_tokens.json",ensure_ascii=False)
    
    

In [15]:
get_transcription_info(dataset_dir/"train.tsv",dataset_dir/"clips")
get_transcription_info(dataset_dir/"validation.tsv",dataset_dir/"clips")
get_transcription_info(dataset_dir/"dev.tsv",dataset_dir/"clips")
get_transcription_info(dataset_dir/"test.tsv",dataset_dir/"clips")

getting audio info about : train


  0%|          | 0/4479 [00:00<?, ?it/s]

100%|██████████| 4479/4479 [09:02<00:00,  8.25it/s]


['।', ',', '।', '?', '।', '।', ',', '!', '।', '।', ',', '।', '।', ',', '।', '।', '।', ',', ',', '।', '.', '.', '.', '.', '.', '.', '!', '?', '।', '.', '.', '.', '।', ':', ',', ':', '।', '-', ',', '।', '।', '।', ',', ',', ',', ',', ',', '।', '।', '?', '।', '?', '।', '।', '।', ':', ',', '.', '.', '.', '।', '।', '।', '।', ',', ':', '।', '।', '।', '।', ',', '?', '।', '।', ':', '।', ',', ',', ',', '?', '।', '।', '।', '।', '?', '!', ',', '।', '?', '।', '।', '!', '।', '।', '।', '।', '?', ',', '?', '‘', '’', '।', '।', '।', ':', ',', '?', '।', 'H', 'a', 'p', 'p', 'y', 'B', 'i', 'r', 't', 'h', 'd', 'a', 'y', ':', '।', '।', '।', '।', '-', '।', '-', '।', '।', ',', ':', '।', ',', ':', '?', ',', '।', ',', ':', ',', '।', ':', '!', '?', '।', '!', '।', '।', '।', ',', '।', ':', '‘', '’', '।', '।', ',', '।', '।', '।', '।', '-', '।', '।', '।', '।', '।', '!', '।', '?', '।', '!', '।', '।', '?', ',', ',', '.', '.', '.', ',', '।', '।', ':', '-', ',', '-', '।', '।', '?', '?', '?', 'P', 'l', 'a', 'y', 'e', 'r',

FileNotFoundError: [Errno 2] No such file or directory: '/media/dataset-harddisk/munikumar/hindi_dataset/common_voice/cv-corpus-13.0-2023-03-09/hi/validation.tsv'